In [72]:
from ngsolve import *
from netgen.occ import *

# 1. 创建一维网格
geo = Segment((0,0,0),(1,0,0))
# help(geo)
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.3))
fmesh = Mesh(OCCGeometry(geo, dim=1).GenerateMesh(maxh=0.3))
help(fmesh)
bds = fmesh.GetBoundaries()
print(bds)

Help on Mesh in module ngsolve.comp object:

class Mesh(pybind11_builtins.pybind11_object)
 |  NGSolve interface to the Netgen mesh. Provides access and functionality
 |  to use the mesh for finite element calculations.
 |
 |  Parameters:
 |
 |  mesh (netgen.Mesh): a mesh generated from Netgen
 |
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  BBBoundaries(...)
 |      BBBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |
 |      Return co dim 3 boundary mesh-region matching the given regex pattern
 |
 |  BBoundaries(...)
 |      BBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |
 |      Return co dim 2 boundary mesh-region matching the given regex pattern
 |
 |  Boundaries(...)
 |      Boundaries(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. Boundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |
 |      Ret

In [84]:
# help(geo.end)
fmesh.ngmesh.SetBCName(0,"start")
fmesh.ngmesh.SetBCName(1,"end")
fmesh=Mesh(fmesh.ngmesh)
bds = fmesh.GetBoundaries()
print(bds)
b1 = fmesh.Boundaries('start')
print(b1)

('start', 'end')


In [85]:
# for i in range(fmesh.nvb):
#     point = fmesh.Point(i)
#     print(f"Vertex {i}: {point}")
fm = fmesh.ngmesh
help(fm)

Help on Mesh in module netgen.libngpy._meshing object:

class Mesh(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  Add(...)
 |      Add(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.MeshPoint) -> netgen.libngpy._meshing.PointId
 |
 |      2. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.Element3D) -> netgen.libngpy._meshing.ElementId3D
 |
 |      3. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.Element2D) -> netgen.libngpy._meshing.ElementId2D
 |
 |      4. Add(self: netgen.libngpy._meshing.Mesh, el: netgen.libngpy._meshing.Element1D, project_geominfo: bool = False) -> netgen.libngpy._meshing.ElementId1D
 |
 |      5. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.Element0D) -> int
 |
 |      6. Add(self:

In [80]:
# print(fm.Elements1D())
print(fm.GetGeometry())

GetSubMesh(...)
 |      GetSubMesh(self: netgen.libngpy._meshing.Mesh, domains: str = '', faces: str = '') -> netgen.libngpy._meshing.Mesh

In [81]:
Draw(mesh)
for v in mesh.vertices:
    print(v,v.point)
for el in mesh.Elements(VOL):
    print("vertices:",el.vertices)
    # print(type(el.vertices[0]))
    print("edges:",el.edges)
    # print(type(el.edges[0]))

V0 (0.0, 0.0)
V1 (1.0, 0.0)
V2 (0.3333333333333334, 0.0)
V3 (0.6666666666666667, 0.0)


In [86]:
# 2. 定义有限元空间
# fes = H1(mesh, order=1, dirichlet=".*")
# fes = H1(mesh, order=1)
fes = H1(fmesh,order=1,dirichlet='start')
print(fes.ndof)
print(fes.FreeDofs())

4
0: 0111


In [37]:
# 3. 定义未知函数和测试函数
u = fes.TrialFunction()
v = fes.TestFunction()

# 4. 定义右端项和双线性形式
f = CoefficientFunction(-2)  # 源项 f(x) = 2
a = BilinearForm(fes)
a += grad(u) * grad(u) * dx  # 泊松方程弱形式

# 5. 定义线性形式 (右端项)
l = LinearForm(fes)
l += f * v * dx

# 6. 应用边界条件
#    u(0)=0, u(1)=1
u_exact = x*x  # 精确解用于边界条件
with TaskManager():
    a.Assemble()
    l.Assemble()

# 7. 创建解向量并求解
gfu = GridFunction(fes)
gfu.Set(u_exact, BND)  # 设置边界条件
r = l.vec.CreateVector()
r.data = l.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * r

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


In [26]:
print(a.mat)

Row 0:   0: 0   2: 0
Row 1:   1: 0   3: 0
Row 2:   0: 0   2: 0   3: 0
Row 3:   1: 0   2: 0   3: 0



In [27]:
print(u)

coef trial-function diffop = Id, real



In [28]:
# 8. 可视化结果
Draw(gfu, mesh, "solution")
# Draw(u_exact - gfu, mesh, "error")

In [31]:
error = sqrt(Integrate((gfu-u_exact)**2, fmesh))
print(error)

0.44721359549995726


In [14]:
help(Draw)

Help on built-in function Draw in module ngsolve.solve:

Draw(...) method of builtins.PyCapsule instance
    Draw(*args, **kwargs)
    Overloaded function.

    1. Draw(cf: ngsolve.fem.CoefficientFunction, mesh: ngsolve.comp.Mesh, name: str, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, draw_vol: bool = True, draw_surf: bool = True, reset: bool = False, title: str = '', number_format: str = '%.3e', unit: str = '', **kwargs) -> None


    Parameters:

    cf : ngsolve.comp.CoefficientFunction
      input CoefficientFunction to draw

    mesh : ngsolve.comp.Mesh
      input mesh

    name : string
      input name

    sd : int
      input subdivisions

    autoscale : bool
      input autscale

    min : float
      input minimum value. Need autoscale = false

    max : float
      input maximum value. Need autoscale = false

    draw_vol : bool
      input draw volume

    draw_surf : bool
      input draw surface

    title : string
      printed on top of c